In [ ]:
import pm4py

log = pm4py.read_xes("BPI Challenge 2017.xes")

In [ ]:
import random
all_cases = list(log["case:concept:name"].unique())
sample_cases = random.choices(all_cases, k=1000)

sampled_log = log[log["case:concept:name"].isin(sample_cases)]
sampled_log

In [ ]:
for case_id, case in sampled_log.groupby("case:concept:name"):
    case = case.sort_values(by='time:timestamp') 
    current_state = "False"
    for idx, row in case.iterrows():
        if row["Accepted"] == True:
            current_state = "True"
        sampled_log.at[idx, "Accepted"] = current_state
altered_log = sampled_log.sort_values(by='case:concept:name') 

In [ ]:
pm4py.write_xes(altered_log, "BPI_Challenge_2017.xes", case_id_key='case:concept:name')

In [ ]:
import ocel
ocel_log = ocel.import_log("BPI_Challenge_2017.xes.jsonocel") # get this from https://gitlab.uni-mannheim.de/processanalytics/uncovering-object-centric-data/-/blob/main/output/evaluation/BPI_Challenge_2017.xes.jsonocel

In [ ]:

# sample_cases
ocel_dict = {
    "ocel:global-event": {
        "ocel:activity": "__INVALID__"
    },
    "ocel:global-object": {
        "ocel:type": "__INVALID__"
    },
    "ocel:global-log": {
        "ocel:attribute-names": [

        ],
        "ocel:object-types": [

        ],
    },
    "ocel:events": {},
    "ocel:objects": {}
}
ocel_dict["ocel:global-log"]["ocel:object-types"] = ocel_log["ocel:global-log"]["ocel:object-types"]
ocel_dict["ocel:global-log"]["ocel:attribute-names"] = ocel_log["ocel:global-log"]["ocel:attribute-names"]

id_to_event = {}
referenced_objs = set()
cnt = 0
for case in sample_cases:
    current_state = "False"
    for key, event in ocel_log["ocel:events"].items():
        if case not in event["ocel:omap"]:
            continue
        #print(type(event))
        event["ocel:vmap"] = dict(event["ocel:vmap"])
        #print(type(event["ocel:vmap"]))
        if "Accepted" in event["ocel:vmap"] and event["ocel:vmap"]["Accepted"] == True:
            current_state = "True"
            print("accepted")
        event["ocel:vmap"]["OfferAccepted"] = current_state
        referenced_objs.update(event["ocel:omap"])
        id_to_event[key] = event
    cnt += 1
    if cnt % 100 == 0:
        print("done with", cnt, "cases")

from collections import OrderedDict
id_to_event = OrderedDict(sorted(id_to_event.items()))
objs_dict = {key: obj for key, obj in ocel_log["ocel:objects"].items()}

ocel_dict["ocel:events"] = dict(id_to_event)
ocel_dict["ocel:objects"] = objs_dict


In [ ]:
ocel.export_log(ocel_dict, "BPI_2017_altered.jsonocel")